In [1]:
import random
import datetime

In [2]:
import pyspark

In [3]:
sc = pyspark.SparkContext()

## 生成100万个字母组成的随机列表

In [4]:
x = [chr(random.randint(65,91)) for i in range(1000000)]

## 生成利用转换算子parallelize将列表转换为RDD

## 生成了一个管道集合RDD，顾名思义，只是生成一个管道，而不是实体

In [8]:
s = datetime.datetime.now()
rdd = sc.parallelize(x)
print(datetime.datetime.now() - s)
print(rdd)

0:00:00.222304


## 执行count这个行动算子，开始提交任务，所以整体执行时间需要5秒

In [11]:
s = datetime.datetime.now()
x2 = rdd.count()
print(datetime.datetime.now() - s)
print(x2)

0:00:05.061927
1000000


## 在rdd算子上面执行一个转换算子map，使每个字母进行一个计数，实际上就是一个转换过程：
### 集合（"w"） ——> 集合("w",1) 

## map算子作为一个转换算子，仅仅记录一个函数过程链，而不发生实际的操作，所以完全没有花费任何执行时间。

In [13]:
s = datetime.datetime.now()
rdd2 = rdd.map(lambda x : (x,1))
print(datetime.datetime.now() - s)
print(rdd2)

0:00:00
PythonRDD[4] at RDD at PythonRDD.scala:48


## 同上，reduceByKey算子，也是一个转换算子，转换的结果是得到一个同key相交的结果，如：

### (("w",1),("w",1),("w",1),("z",1),("z",1))  ——>
### (("w",3),("z",2))

## 同样，reduceByKey算子也不会触发执行，所以也没有花费执行时间

In [16]:
s = datetime.datetime.now()
rdd3 = rdd2.reduceByKey(lambda a,b : a+b)
print(datetime.datetime.now() - s)
print(rdd3)

0:00:00.034910
PythonRDD[15] at RDD at PythonRDD.scala:48


## 最后调用collect算子，这是一个action算子，开始执行全环节运行，全部函数链执行。

In [17]:
s = datetime.datetime.now()
res = rdd3.collect()
print(datetime.datetime.now() - s)
print(res)

0:00:09.629967
[('[', 36920), ('O', 36954), ('N', 37148), ('J', 36862), ('B', 36882), ('H', 36955), ('W', 37045), ('C', 37055), ('R', 36978), ('X', 37271), ('V', 36881), ('Z', 37272), ('I', 36911), ('K', 37436), ('L', 36595), ('A', 36775), ('T', 37037), ('M', 37113), ('Q', 36969), ('U', 37495), ('F', 37137), ('S', 37112), ('G', 36919), ('Y', 36865), ('D', 37267), ('P', 36932), ('E', 37214)]
